In [2]:
import s3fs
import numpy as np
import pandas as pd

from mvpa2.datasets.mri import fmri_dataset
from mvpa2.clfs.meta import SplitClassifier
from mvpa2.generators.partition import NFoldPartitioner
from mvpa2 import cfg
from mvpa2.clfs.svm import libsvm
import mvpa2.datasets as md
from mvpa2.featsel.base import SensitivityBasedFeatureSelection
from mvpa2.clfs.warehouse import OneWayAnova
from mvpa2.clfs.warehouse import FixedNElementTailSelector
from mvpa2.clfs.meta import FeatureSelectionClassifier
from mvpa2.datasets.mri import map2nifti

from mvpa2.mappers.fx import maxofabs_sample

/srv/conda/envs/notebook/lib/python3.7/site-packages/mvpa2/datasets/base.py:465: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def __init__(self, shape=None, sid=None, fid=None, dtype=np.float):


In [3]:
## Load the data file names
fs = s3fs.S3FileSystem(anon=True)
ll = fs.ls('natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR')
sess_beta_list = [l for l in ll if l.split('/')[6].startswith("betas") and l.endswith("nii.gz")]

behav_data = pd.read_csv('subj01/behav/final_response.tsv', sep='\t')

In [4]:
sess_beta_list

['natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session01.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session02.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session03.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session04.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session05.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session06.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session07.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session08.nii.gz',
 'natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fi

In [5]:
behav_data

,SUBJECT,SESSION,RUN,TRIAL,73KID,10KID,TIME,ISOLD,ISCORRECT,RT,...,ISOLDCURRENT,ISCORRECTCURRENT,TOTAL1,TOTAL2,BUTTON,MISSINGDATA,people_count,animal_count,shared1000,interaction_type
0,1,1,1,1,46003,626,0.505082,0,1.0,803.529781,...,0,1.0,1,0,1.0,0,0,14,True,NaN
1,1,18,1,53,46003,626,133.667932,1,0.0,1749.405303,...,0,1.0,1,0,1.0,0,0,14,True,NaN
2,1,35,6,24,46003,626,239.717843,1,0.0,1202.503384,...,0,1.0,1,0,1.0,0,0,14,True,NaN
3,1,1,1,2,61883,5013,0.505128,0,1.0,972.261383,...,0,1.0,1,0,1.0,0,9,0,False,NaN
4,1,13,2,26,61883,5013,91.617241,1,1.0,2411.506844,...,0,0.0,1,1,2.0,0,9,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,40,9,46,52260,7543,262.615194,1,1.0,701.858252,...,1,1.0,0,1,2.0,0,13,7,False,NaN
29996,1,40,11,30,52260,7543,262.623441,1,1.0,579.064407,...,1,1.0,0,1,2.0,0,13,7,False,NaN
29997,1,40,9,24,37847,8122,262.614083,0,0.0,902.196521,...,0,0.0,0,1,2.0,0,1,0,False,NaN
29998,1,40,10,13,37847,8122,262.618065,1,1.0,657.601719,...,1,1.0,0,1,2.0,0,1,0,False,NaN


In [6]:
### Load the data
# load the first session
sess = 1
sess_fname = sess_beta_list[0]
# load the session data
fs.get(sess_fname, "tmp.nii.gz")
sess_data = fmri_dataset("tmp.nii.gz", mask = "TPJfunc.nii")

# extract the behavioral data for the session
tmp_behav = behav_data[behav_data['SESSION'] == sess] 

# only take the trials that have an interaction_type (i.e., not null)
indices = tmp_behav['interaction_type'].notnull().tolist()

# take the beta values for those trials
all_sess_ds = sess_data[indices, :]

# add the target (interaction_type) and chunk (session) information to the dataset
targets = tmp_behav[indices]['interaction_type'].tolist()
chunks = tmp_behav[indices]['SESSION']

all_sess_ds.sa['targets'] = targets
all_sess_ds.sa['chunks'] = chunks

# add an attribute that determines whether the session should be included in the training or test dataset
if (sess) >=36:
    all_sess_ds.sa['train_or_test'] = ['test'] * len(all_sess_ds)
else:
    all_sess_ds.sa['train_or_test'] = ['train'] * len(all_sess_ds)

# then concatenate the other sessions to the first session
for sess, sess_fname in enumerate(sess_beta_list[1:3]): 
    print(sess+2)
    print(sess_fname)
    # load the session data
    fs.get(sess_fname, "tmp.nii.gz")
    sess_data = fmri_dataset("tmp.nii.gz", mask = "TPJfunc.nii")

    # extract the behavioral data for the session
    tmp_behav = behav_data[behav_data['SESSION'] == (sess+2)] 

    # only take the trials that have an interaction_type (i.e., not null)
    indices = tmp_behav['interaction_type'].notnull().tolist()

    # take the beta values for those trials
    tmp_ds = sess_data[indices, :]

    # add the target (interaction_type) and chunk (session) information to the dataset
    targets = tmp_behav[indices]['interaction_type'].tolist()
    chunks = tmp_behav[indices]['SESSION']
    tmp_ds.sa['targets'] = targets
    tmp_ds.sa['chunks'] = chunks
    
    # add an attribute that determines whether the session should be included in the training or test dataset
    if (sess+2) >=36:
        tmp_ds.sa['train_or_test'] = ['test'] * len(tmp_ds)
    else:
        tmp_ds.sa['train_or_test'] = ['train'] * len(tmp_ds)
    
    # concatenate the tmp_ds to the all_sess_ds
    all_sess_ds = md.vstack((all_sess_ds, tmp_ds))

/srv/conda/envs/notebook/lib/python3.7/site-packages/mvpa2/datasets/mri.py:78: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data, header = img.get_data(), img.header


 * Please note: warnings are printed only once, but underlying problem might occur many times *
2
natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session02.nii.gz
3
natural-scenes-dataset/nsddata_betas/ppdata/subj01/func1pt8mm/betas_fithrf_GLMdenoise_RR/betas_session03.nii.gz


In [7]:
print(f"Dataset shape is: {all_sess_ds.shape}")

Dataset shape is: (37, 1000)


In [8]:
all_sess_ds.samples

array([[  -16,    82,    49, ...,  -332,  -546,   591],
       [ -104,    60,  -352, ...,  -476,   468,  -368],
       [   70,   655,  -552, ...,   -23,     4,   417],
       ...,
       [  202,   -90,   -52, ...,  2117,  -207, -3702],
       [ -213,   286,  -652, ...,   663,   -57, -5568],
       [  262,   -29,    -7, ...,  -833,   171,  1513]], dtype=int16)

In [9]:
## convert the integer values to percent change (according to https://cvnlab.slite.com/p/6CusMRYfk0/Functional-data-NSD)
normalized_all_sess_ds = all_sess_ds.copy()
normalized_all_sess_ds.samples = normalized_all_sess_ds.samples.astype('float')/300
normalized_all_sess_ds.samples

array([[-5.33333333e-02,  2.73333333e-01,  1.63333333e-01, ...,
        -1.10666667e+00, -1.82000000e+00,  1.97000000e+00],
       [-3.46666667e-01,  2.00000000e-01, -1.17333333e+00, ...,
        -1.58666667e+00,  1.56000000e+00, -1.22666667e+00],
       [ 2.33333333e-01,  2.18333333e+00, -1.84000000e+00, ...,
        -7.66666667e-02,  1.33333333e-02,  1.39000000e+00],
       ...,
       [ 6.73333333e-01, -3.00000000e-01, -1.73333333e-01, ...,
         7.05666667e+00, -6.90000000e-01, -1.23400000e+01],
       [-7.10000000e-01,  9.53333333e-01, -2.17333333e+00, ...,
         2.21000000e+00, -1.90000000e-01, -1.85600000e+01],
       [ 8.73333333e-01, -9.66666667e-02, -2.33333333e-02, ...,
        -2.77666667e+00,  5.70000000e-01,  5.04333333e+00]])

In [10]:
## The classification procedure
num_features = 300
fsel = SensitivityBasedFeatureSelection(OneWayAnova(), 
                                        FixedNElementTailSelector(num_features, mode='select', tail='upper'))
clf = libsvm.SVM()
clf = FeatureSelectionClassifier(clf, fsel)
sclf = SplitClassifier(clf, NFoldPartitioner(cvtype=1), enable_ca=['stats'])
cv_sensana = sclf.get_sensitivity_analyzer()

In [11]:
## Apply the classifier
sens = cv_sensana(normalized_all_sess_ds)
sens_comb = sens.get_mapped(maxofabs_sample())

/srv/conda/envs/notebook/lib/python3.7/site-packages/mvpa2/misc/attrmap.py:171: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  num = np.empty(attr.shape, dtype=np.int)


In [12]:
print(cv_sensana.clf.ca.stats.matrix)

[[5 4 3]
 [5 4 7]
 [1 7 1]]


In [13]:
print(cv_sensana.clf.ca.stats.as_string(description=True))

----------.
predictions\targets   Joint attention    Social interaction   Unrelated people
            `------  ------------------  ------------------  ------------------   P'    N'  FP FN  PPV  NPV  TPR  SPC  FDR  MCC   F1
  Joint attention            5                   4                   3            12    11   7  6 0.42 0.45 0.45 0.42 0.58 -0.09 0.43
 Social interaction          5                   4                   7            16    17  12 11 0.25 0.35 0.27 0.33 0.75 -0.36 0.26
  Unrelated people           1                   7                   1            9     19   8 10 0.11 0.47 0.09 0.53 0.89 -0.32  0.1
Per target:          ------------------  ------------------  ------------------
         P                   11                  15                  11
         N                   26                  22                  26
         TP                  5                   4                   1
         TN                  5                   6                   9
Summary 

In [19]:
## Plot the weights outputted by the classifier
senstime = fs.ls('natural-scenes-dataset/nsddata_timeseries/ppdata/subj01/func1pt8mm/timeseries/timeseries_session01_run01.nii.gz')
sens = senstime[0]
fs.get(sens, "sens.nii.gz")
sensd = fmri_dataset("sens.nii.gz", mask="TPJfunc.nii")

sensd.shape

nimg = map2nifti(sensd, sens_comb)
nimg.to_filename('SVM.nii')

/srv/conda/envs/notebook/lib/python3.7/site-packages/mvpa2/datasets/mri.py:78: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data, header = img.get_data(), img.header
